In [8]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [9]:
math=pd.read_csv('Maths.csv')

In [15]:
math.head(5)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,no,no,4,3,4,1,1,3,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,no,5,3,3,1,1,3,4,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,no,4,3,2,2,3,3,10,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,3,2,2,1,1,5,2,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,no,no,4,3,2,1,2,5,4,10


In [14]:
math.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 31 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   school      395 non-null    object
 1   sex         395 non-null    object
 2   age         395 non-null    int64 
 3   address     395 non-null    object
 4   famsize     395 non-null    object
 5   Pstatus     395 non-null    object
 6   Medu        395 non-null    int64 
 7   Fedu        395 non-null    int64 
 8   Mjob        395 non-null    object
 9   Fjob        395 non-null    object
 10  reason      395 non-null    object
 11  guardian    395 non-null    object
 12  traveltime  395 non-null    int64 
 13  studytime   395 non-null    int64 
 14  failures    395 non-null    int64 
 15  schoolsup   395 non-null    object
 16  famsup      395 non-null    object
 17  paid        395 non-null    object
 18  activities  395 non-null    object
 19  nursery     395 non-null    object
 20  higher    

In [26]:
lister=math.dtypes==object
obj_cols=lister[lister==True].index.to_list()

In [27]:
le=LabelEncoder()
for col in obj_cols:
    math[col]=le.fit_transform(math[col])

In [28]:
math

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G3
0,0,0,18,1,0,0,4,4,0,4,...,0,0,4,3,4,1,1,3,6,6
1,0,0,17,1,0,1,1,1,0,2,...,1,0,5,3,3,1,1,3,4,6
2,0,0,15,1,1,1,1,1,0,2,...,1,0,4,3,2,2,3,3,10,10
3,0,0,15,1,0,1,4,2,1,3,...,1,1,3,2,2,1,1,5,2,15
4,0,0,16,1,0,1,3,3,2,2,...,0,0,4,3,2,1,2,5,4,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,1,1,20,1,1,0,2,2,3,3,...,0,0,5,5,4,4,5,4,11,9
391,1,1,17,1,1,1,3,1,3,3,...,1,0,2,4,5,3,4,2,3,16
392,1,1,21,0,0,1,1,1,2,2,...,0,0,5,5,3,3,3,3,3,7
393,1,1,18,0,1,1,3,2,3,2,...,1,0,4,4,1,3,4,5,0,10


In [30]:
X=math.drop(columns='G3')
y=math.G3

In [31]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=42)

In [55]:
# Using Decision Tree Regressor
dt=DecisionTreeRegressor()
params={'max_depth':[2,3,4,5,6,7,8,9,10],
       'min_samples_leaf':[3,4,5,6,7,8,9,10,12,13,14,15,16,17,18,19,20]}
dt_grid=GridSearchCV(dt,param_grid=params,cv=5,scoring='r2')
dt_grid.fit(X_train,y_train)
print('Score is ',dt_grid.score(X_train,y_train))
print('Best Params are ',dt_grid.best_params_)
from sklearn.metrics import mean_absolute_error as mae
preds=dt_grid.predict(X_test)
mae(preds,y_test)/y_test.median()

Score is  0.42060730155891757
Best Params are  {'max_depth': 8, 'min_samples_leaf': 13}


0.407988846888283

In [57]:
pd.DataFrame({'preds':preds,'Actuals':y_test}).head(5)

,preds,Actuals
78,8.500000,10
371,9.214286,12
248,10.105263,5
55,11.671429,10
390,8.500000,9


In [72]:
# Using Random forest Regressor
from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor()
params={'max_depth':[4,5,6,7,8,9,10,11,12,13],
       'n_estimators':[2,4,6,8,10,12,14,16],
       'max_features':['sqrt','log2']}
rf_grid=GridSearchCV(rf,param_grid=params,cv=5,scoring='r2')
rf_grid.fit(X_train,y_train)
print('Score is ',rf_grid.score(X_train,y_train))
print('Best Params are ',rf_grid.best_params_)

Score is  0.7560964920282492
Best Params are  {'max_depth': 9, 'max_features': 'sqrt', 'n_estimators': 12}


In [73]:
preds=rf_grid.predict(X_test)
res=pd.DataFrame({'preds':preds,'Actuals':y_test})
from sklearn.metrics import mean_absolute_error as mae
mae(preds,y_test)/y_test.median()


0.35403463051865985

In [74]:
res

,preds,Actuals
78,7.420635,10
371,7.191667,12
248,8.665824,5
55,10.868627,10
390,11.250000,9
223,6.994786,13
42,13.014096,18
234,11.832765,6
316,8.158333,0
116,10.514825,14


In [66]:
discrete=X_train.dtypes==int

In [75]:
mi=mutual_info_regression(X_train,y_train,discrete_features=discrete)
sel_features=pd.Series(mi,index=X_train.columns).sort_values(ascending=False).head(10).to_list()

In [63]:
discrete

Index(['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
       'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime',
       'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',
       'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc',
       'Walc', 'health', 'absences'],
      dtype='object')